# Agent Tools

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Load API keys
openai_api_key = os.getenv("OPENAI_API_KEY")
brave_api_key = os.getenv("BRAVE_API_KEY")

In [2]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.agents.load_tools import get_all_tool_names
from langchain_openai import ChatOpenAI

In [8]:
# For tools/agents, use temeprature=0 to get deterministic results
llm = ChatOpenAI(temperature=0)

In [3]:
# Tools:
# # https://python.langchain.com/v0.2/docs/integrations/tools/
# For most of them we need to set an API key
# Check each integration page for instructions
# Here the SerpAPI is used: Search Engine Results API
# https://python.langchain.com/v0.2/docs/integrations/tools/serpapi/
# https://serpapi.com/
# from langchain_community.utilities import SerpAPIWrapper
# However, I decided to choose the Brave Search API
# https://python.langchain.com/v0.2/docs/integrations/tools/brave_search/
# https://api.search.brave.com
from langchain_community.tools import BraveSearch

In [4]:
search_tool = BraveSearch.from_api_key(
    api_key=brave_api_key,
    search_kwargs={"count": 3}
)

In [5]:
type(search_tool)

langchain_community.tools.brave_search.tool.BraveSearch

In [6]:
search_tool.run("obama middle name")

'[{"title": "Barack Obama - Wikipedia", "link": "https://en.wikipedia.org/wiki/Barack_Obama", "snippet": "In January 2007, he left the ... subcommittee on European Affairs. As a member of the Senate Foreign Relations Committee, <strong>Obama</strong> made official trips to Eastern Europe, the <strong>Middle</strong> East, Central Asia, and Africa...."}, {"title": "Obama\'s Middle Name - My Last Name - is \'Hussein.\' So? -", "link": "https://www.cair.com/cair_in_the_news/obamas-middle-name-my-last-name-is-hussein-so/", "snippet": "Many Americans understand that common names don\\u2019t only come in the form of a \\u201cSmith\\u201d or a \\u201cJohnson.\\u201d Perhaps, they have a neighbor, mechanic or teacher named Hussein. Or maybe they\\u2019ve seen fashion designer Hussein Chalayan in the pages of Vogue or recall <strong>King Hussein</strong>, our ally in ..."}, {"title": "What\'s up with Obama\'s middle name? - Quora", "link": "https://www.quora.com/Whats-up-with-Obamas-middle-name

In [14]:
# We load the LC math tool and add the Brave search tool to the list
tools = load_tools(["llm-math"], llm=llm) # list
tools = tools + [search_tool]

In [16]:
# We create a Zero-shot Agent
# WARNING: initialize_agent is deprecated...
# Instead, we should use new agent constructor methods:
# create_react_agent, create_json_agent, create_structured_chat_agent, ...
agent = initialize_agent(
    tools, # Brave-Search
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True # check what is happening inside
)

c:\Users\A200239740\AppData\Local\anaconda3\envs\llms\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [18]:
agent.run("What is the year Albert Einstein was born? What is this year multiplied by 5?")



> Entering new AgentExecutor chain...
I can use a calculator to find the year Albert Einstein was born and then multiply it by 5.
Action: Calculator
Action Input: 1879, 5
Observation: Answer: 1884
Thought:I now know the final answer
Final Answer: 1884

> Finished chain.


'1884'

In [19]:
# New interfaces
# https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.agents
# https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html#langchain.agents.react.agent.create_react_agent
from langchain.agents.react.agent import create_react_agent
from langchain.agents import AgentExecutor

In [20]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [21]:
agent = create_react_agent(
    llm,
    tools,
    prompt
)

In [22]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [23]:
agent_executor.invoke({"input": "What is the year Albert Einstein was born? What is this year multiplied by 5?"})

{'input': 'What is the year Albert Einstein was born? What is this year multiplied by 5?',
 'output': '1884'}